In [1]:
import pickle
from apply_dichotomized_prediction_loocv_and_save import *
#     def apply_loocv_and_save(
#     brain_data_filepath = '../data/Brain_Data_2sns_60subs.pkl',
#     train_test_markers_filepath = "../data/train_test_markers_20210601T183243.csv",
#     results_filepath="",
#     subjs_to_use = None #set this to get a subset, otherwise use all of them.
# ):
    
apply_dichotomized_prediction_loocv_and_save(
    results_filepath="../data/cv_train_test_ns_5subjs_outer_n_loocv.pkl",
    brain_data_filepath = '../data/Brain_Data_ns_5subs.pkl',
    train_test_markers_filepath = "../data/train_test_markers_20210601T183243.csv"
    
)


python initialized for apply_loocv_and_save


/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


4
checked for intersection and no intersection between the brain data and the subjects was found.
there were 5 subjects overlapping between the subjects marked for train data and the training dump file itself.
0    205
1    115
Name: response, dtype: int64
0    205
1    115
Name: response, dtype: int64
False    320
Name: response, dtype: int64
320
320
295.6 MiB
1.1 GiB
starting LeaveOneOut
finished preprocessing
Groups are the same.
fold 1 of 5
In order to test on a training group of 4 items, holding out the following subjects:['DEV010']. prepping fold data.... regressing.... 1.7 GiB. trying regressor 1 of 1. predicting. test score was:. -0.33333333333333326
fold 2 of 5
In order to test on a training group of 4 items, holding out the following subjects:['DEV009']. prepping fold data.... regressing.... 1.7 GiB. trying regressor 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. -0.6288441145281018
fold 3 of 5
In order to test on a training group of 4 items, holding out the following subjects:['DEV006']. prepping fold data.... regressing.... 1.7 GiB. trying regressor 1 of 1. 

...

predicting. test score was:. -0.3852813852813852
fold 4 of 5
In order to test on a training group of 4 items, holding out the following subjects:['DEV001']. prepping fold data.... regressing.... 1.7 GiB. trying regressor 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. -0.7622027534418023
fold 5 of 5
In order to test on a training group of 4 items, holding out the following subjects:['DEV005']. prepping fold data.... regressing.... 1.7 GiB. trying regressor 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. -0.5133004926108375
-0.33333333333333326
-0.33333333333333326
finished learning
saved.


In [2]:
print("python initialized for apply_loocv_and_save")


import pandas as pd
import numpy as np
import os
import nltools as nlt
import nilearn as nil
import nibabel as nib
import warnings
import glob
import random
import pickle
import dev_wtp_io_utils
import gc #garbage collection
from nilearn import plotting
from dev_wtp_io_utils import cv_train_test_sets, asizeof_fmt
from sklearn.model_selection import KFold,GroupKFold,LeaveOneOut
import os, warnings
import pickle

cpus_available = int(os.getenv('CPUS_PER_TASK'))
#custom thing I have set in my jupyter notebook task.
print(cpus_available)


python initialized for apply_loocv_and_save
4


In [3]:
results_filepath="../data/cv_train_test_ns_5subjs_outer_n_loocv.pkl"
brain_data_filepath = '../data/Brain_Data_ns_5subs.pkl'
train_test_markers_filepath = "../data/train_test_markers_20210601T183243.csv"

In [4]:
test_train_set = pd.read_csv(train_test_markers_filepath)

with open(brain_data_filepath, 'rb') as pkl_file:
    Brain_Data_allsubs = pickle.load(pkl_file)

dev_wtp_io_utils.check_BD_against_test_train_set(Brain_Data_allsubs,test_train_set)


checked for intersection and no intersection between the brain data and the subjects was found.
there were 5 subjects overlapping between the subjects marked for train data and the training dump file itself.


In [5]:
X= Brain_Data_allsubs.X

In [24]:
wsr_means = pd.DataFrame(X.groupby(['wave','subject','run']).response.mean()).reset_index()
wsr_means = wsr_means.rename(columns={'response':'response_mean'})
wsr_sds = pd.DataFrame(X.groupby(['wave','subject','run']).response.std()).reset_index()
wsr_sds = wsr_sds.rename(columns={'response':'response_sd'})

In [32]:
X_augmented = X[['run','wave','subject','response']].copy().merge(wsr_means).merge(wsr_sds)

In [33]:
response_norm = (X_augmented.response-X_augmented.response_mean)/X_augmented.response_sd

In [34]:
response_norm

0      1.175024
1      0.130558
2      0.130558
3     -0.913908
4      1.175024
         ...   
315   -1.009009
316    1.614415
317   -0.134535
318         NaN
319         NaN
Length: 320, dtype: float64

In [35]:
def transform_normalize(X):
    wsr_means = pd.DataFrame(X.groupby(['wave','subject','run']).response.mean()).reset_index()
    wsr_means = wsr_means.rename(columns={'response':'response_mean'})
    wsr_sds = pd.DataFrame(X.groupby(['wave','subject','run']).response.std()).reset_index()
    wsr_sds = wsr_sds.rename(columns={'response':'response_sd'})
    
    X_augmented = X[['run','wave','subject','response']].copy().merge(wsr_means).merge(wsr_sds)

    response_norm = (X_augmented.response-X_augmented.response_mean)/X_augmented.response_sd
    
    #now we need to group by 
    return(response_norm)
    
transform_normalize(Brain_Data_allsubs.X)

0      1.175024
1      0.130558
2      0.130558
3     -0.913908
4      1.175024
         ...   
315   -1.009009
316    1.614415
317   -0.134535
318         NaN
319         NaN
Length: 320, dtype: float64

In [36]:
import pickle
from apply_loocv_and_save import *

    
apply_loocv_and_save(
    results_filepath="../data/cv_train_test_ns_5subjs_outer_n_loocv.pkl",
    brain_data_filepath = '../data/Brain_Data_ns_5subs.pkl',
    train_test_markers_filepath = "../data/train_test_markers_20210601T183243.csv",
    response_transform_func =transform_normalize
    
)


python initialized for apply_loocv_and_save
4
checked for intersection and no intersection between the brain data and the subjects was found.
there were 5 subjects overlapping between the subjects marked for train data and the training dump file itself.
 0.000000    9
 0.091902    9
-0.824040    8
-0.835937    8
-0.484123    8
            ..
 2.219490    1
 1.871942    1
-0.098346    1
 0.866025    1
 0.727094    1
Length: 76, dtype: int64
 0.000000    9
 0.091902    9
-0.824040    8
-0.835937    8
-0.484123    8
            ..
 2.219490    1
 1.871942    1
-0.098346    1
 0.866025    1
 0.727094    1
Length: 76, dtype: int64
test_train_set: 9549
pkl_file: 168
response_transform_func: 136
results_filepath: 98
train_test_markers_filepath: 95
brain_data_filepath: 80
sys: 72
Brain_Data_allsubs: 48
subjs_to_use: 16
False    307
True      13
dtype: int64
307
320
284.3 MiB
1.0 GiB
starting LeaveOneOut
finished preprocessing
Groups are the same.
using default regressor. fold 1 of 5
In order t

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. 0.07746336565448064
fold 3 of 5
In order to test on a training group of 4 items, holding out the following subjects:['DEV006']. prepping fold data.... regressing.... 1.7 GiB. trying regressor 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. 0.2231032326643425
fold 4 of 5
In order to test on a training group of 4 items, holding out the following subjects:['DEV001']. prepping fold data.... regressing.... 1.6 GiB. trying regressor 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. 0.34048640774154815
fold 5 of 5
In order to test on a training group of 4 items, holding out the following subjects:['DEV005']. prepping fold data.... regressing.... 1.6 GiB. trying regressor 1 of 1. 

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/decoding/decoder.py:143: UserWarning: Use a custom estimator at your own risk of the process not working as intended.
  warnings.warn('Use a custom estimator at your own risk '


predicting. test score was:. 0.2120862326740096
0.29401979060088
0.29401979060088
finished learning
saved.
